In [25]:
#conflict
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
import pandas as pd
from pandas import DataFrame as df
from mpld3 import plugins, fig_to_html, save_html, fig_to_dict
from tqdm import tqdm
import numpy as np
import json
from tensorflow.keras.preprocessing.text import text_to_word_sequence
#character, setting
import numpy as np
import gensim
import nltk
import re
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd
from nltk.tokenize import sent_tokenize
import multiprocessing
import os
from pathlib import Path
import io
from gensim.models import Phrases
from textblob import TextBlob
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models, similarities

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cacki\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\cacki\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [26]:
from enum import Enum


class ContainsBio(Enum):
    YES = 1
    NO = 0
    MAYBE = 2


def human_contains_pronouns(bio):
    perfect_matches = ['he/him', 'she/her', 'they/them', 'ze', 'hir']
    probably_matches = ['they', 'he', 'her', 'him', 'her', 'pronouns']

    words = re.findall(r"[\w'/]+", bio.lower())
    for match in perfect_matches:
        if match in words:
            return ContainsBio.YES

    for match in probably_matches:
        if match in words:
            return ContainsBio.MAYBE
    return ContainsBio.NO

In [27]:
input_text= """A window into the soul.For most people, this would be the eyes. The eyes cannot lie; they often tell more about a person's emotions than their words. What distinguishes a fake smile from a genuine one? The eyes. What shows sadness? The eyes. What gives away a liar? The eyes.But are the eyes the only window into the soul?Recently, I began painting with watercolors. With watercolors, there is no turning back: if one section is too dark, it is nearly impossible to lighten the area again. Every stroke must be done purposefully, every color mixed to its exact value.I laid my materials before me, preparing myself for the worst. I checked my list of supplies, making sure my setup was perfect.I wet my brush, dipped it into some yellow ochre, and dabbed off the excess paint. Too little water on my brush. I dipped my brush back into my trusty water jar; the colors swirled beautifully, forming an abstract art piece before my eyes. \u2014It's a shame that I couldn't appreciate it.I continued mixing colors to their exact value. More alizarin crimson. More water. More yellow ochre. Less water. More phthalo blue. The cycle continued. Eventually, I was satisfied. The colors looked good, there was enough contrast between facial features, and the watercolors stayed inside the lines.Craving feedback, I posted my art to Snapchat. I got a few messages such as 'wow' and 'pretty,' but one message stood out. 'You were anxious with this one, huh? Anyways, love the hair!'I was caught off guard. Was it a lucky guess? Did they know something I didn't? I immediately responded: 'Haha, how could you tell?' No response.What I didn't know at the time was that my response would come a few months later while babysitting. Since the girl I was babysitting loved art, I took out some Crayola watercolors and some watercolor paper for her to play with. After I went to the bathroom and came back, the watercolors were doused with water. 'You were impatient with this one, huh? Anyways, love the little dog you drew!'The little girl looked up at me, confused. 'How could you tell?' 'You used a lot of water for a brighter color, but you couldn't wait for it to slowly soak in.''Oh.'Now, I would be lying if I said I realized the connection between the two events immediately.Instead, I made the connection when I decided to sit down one day and objectively critique my art. The piece that I once loved now seemed like a nervous wreck: the paper was overworked, the brushstrokes were undecided, the facial features blended together, and each drop of water was bound inside the lines as if it was a prisoner in a cage.From then on, I started noticing pieces of personality in additional creations surrounding me: website designs, solutions to math problems, code written for class, and even the preparation of a meal.When I peer around at people's projects during Code Club, I notice the clear differences between their code. Some people break it up by commenting in every possible section. Others breeze through the project, not caring to comment or organize their code. I could also see clear differences in personalities when our club members began coding the Arduino for the first time. Some followed the tutorials to the letter, while others immediately started experimenting with different colored LEDs and ways of wiring the circuit.It became clear to me that, as humans, we leave pieces of our souls in everything we do, more than we intend to. If we entertain this thought, perhaps the key to better understanding others around us is simply noticing the subtler clues under our noses?Perhaps there are endless windows to the soul, and we simply need to peer through them. I shakily rose my hand. 'We should create workshops of our own,' I suggested.I got a few strange looks. 'It's a good idea, but it's too much work.' 'We just don't have enough free time to make it work.' 'Maybe we could, but I don't know how to make workshops.' My suggestion was shot down. I shuffled in my seat. 'I could make them.' A few people stared at me in disbelief. I glanced over at the club advisor, Mr. C, nervous to hear his response.'If you're willing to take on the work, we can try it.' Mr. C replied. And so I embarked on my quest. I researched different workshops on the internet, learning the information myself at first. Then, I transitioned into creating workshops of my own, making sure that the information was easy to understand for even a beginner. I was exhausted; my first workshop took 16 cumulative hours to create."""

In [28]:
humnan_pronoun = human_contains_pronouns(input_text)
humnan_pronoun

<ContainsBio.MAYBE: 2>

In [29]:
#Anaphoric reference

def pronouns_Anaphoric(text):

    essay_input_corpus = str(text) #문장입력
    essay_input_corpus = essay_input_corpus.lower()#소문자 변환

    sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
    total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
    total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
    
    split_sentences = []
    for sentence in sentences:
        processed = re.sub("[^a-zA-Z]"," ", sentence)
        words = processed.split()
        split_sentences.append(words)

    skip_gram = 1
    workers = multiprocessing.cpu_count()
    bigram_transformer = Phrases(split_sentences)

    model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

    model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
    
    #모델 설계 완료

    #OVERLAP 단어들을 리스트에 넣어서 필터로 만들고
    character_list = ['all','another','any','anybody','anyone','anything','as','aught','both','each'
                    ,'each other','either','enough','everybody','everyone','everything','few','he','her','hers','herself','him','his','I'
                    ,'idem','it','its','itself','many','me','mine','most','my','myself','naught','neither','no one','nobody','none'
                    ,'nothing','nought','one','one another','other','others','ought','our','ours','ourself','ourselves','several'
                    ,'she','some','somebody','someone','something','somewhat','such','suchlike','that','thee','their','theirs'
                    ,'theirself','theirselves','them','themself','themselves','there','these','they','thine','this','those','thou'
                    ,'thy','thyself','us','we','what','whatever','whatnot','whatsoever','whence','where','whereby','wherefrom','wherein'
                    ,'whereinto','whereof','whereon','wherever','wheresoever','whereto','whereunto','wherewith','wherewithal','whether'
                    ,'which','whichever','whichsoever','who','whoever','whom','whomever','whomso','whomsoever','whose','whosever','whosesoever'
                    ,'whoso','whosoever','ye','yon','yonder','you','your','yours','yourself','yourselves']
    
    ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
    #우선 토큰화한다.
    retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
    token_input_text = retokenize.tokenize(essay_input_corpus)
    #print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
    #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
    filtered_chr_text = []
    for k in token_input_text:
        for j in character_list:
            if k == j:
                filtered_chr_text.append(j)
    
    #print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
    
    filtered_chr_text_ = set(filtered_chr_text) #중복제거
    filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
    #print (filtered_chr_text__) # 중복값 제거 확인
    
    for i in filtered_chr_text__:
        ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
    
    pronouns_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
    pronouns_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수
        
    result_pronouns_ratio = round(pronouns_total_count/total_words * 100, 2)

    #return result_pronouns_ratio, total_sentences, total_words, pronouns_total_count, char_count_, ext_sim_words_key
    return result_pronouns_ratio

In [30]:

anaphoric_reference = pronouns_Anaphoric(input_text)
anaphoric_reference

C:\Users\cacki\anaconda3\envs\py37pytorch\lib\site-packages\ipykernel_launcher.py:59: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).


12.23

In [31]:
#동사, 명사 추출

In [36]:
text = nltk.word_tokenize(input_text)
pos_tagged = nltk.pos_tag(text)
pos_tagged


[('A', 'DT'),
 ('window', 'NN'),
 ('into', 'IN'),
 ('the', 'DT'),
 ('soul.For', 'NN'),
 ('most', 'JJS'),
 ('people', 'NNS'),
 (',', ','),
 ('this', 'DT'),
 ('would', 'MD'),
 ('be', 'VB'),
 ('the', 'DT'),
 ('eyes', 'NNS'),
 ('.', '.'),
 ('The', 'DT'),
 ('eyes', 'NNS'),
 ('can', 'MD'),
 ('not', 'RB'),
 ('lie', 'VB'),
 (';', ':'),
 ('they', 'PRP'),
 ('often', 'RB'),
 ('tell', 'VBP'),
 ('more', 'JJR'),
 ('about', 'IN'),
 ('a', 'DT'),
 ('person', 'NN'),
 ("'s", 'POS'),
 ('emotions', 'NNS'),
 ('than', 'IN'),
 ('their', 'PRP$'),
 ('words', 'NNS'),
 ('.', '.'),
 ('What', 'WP'),
 ('distinguishes', 'VBZ'),
 ('a', 'DT'),
 ('fake', 'JJ'),
 ('smile', 'NN'),
 ('from', 'IN'),
 ('a', 'DT'),
 ('genuine', 'JJ'),
 ('one', 'CD'),
 ('?', '.'),
 ('The', 'DT'),
 ('eyes', 'NNS'),
 ('.', '.'),
 ('What', 'WP'),
 ('shows', 'VBZ'),
 ('sadness', 'NN'),
 ('?', '.'),
 ('The', 'DT'),
 ('eyes', 'NNS'),
 ('.', '.'),
 ('What', 'WP'),
 ('gives', 'VBZ'),
 ('away', 'RP'),
 ('a', 'DT'),
 ('liar', 'NN'),
 ('?', '.'),
 ('The'

In [40]:
nouns = list(filter(lambda x:x[1]=='NN',pos_tagged))
nouns_no = len(nouns)
nouns_no

105

In [43]:
vbzs = list(filter(lambda x:x[1]=='VBZ',pos_tagged))
vbzs_no  = len(vbzs)
vbzs_no

10

In [ ]:
#Connectives analysis
def connecitve_words_ratio(text):

    essay_input_corpus = str(text) #문장입력
    essay_input_corpus = essay_input_corpus.lower()#소문자 변환

    sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
    total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
    total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
    
    split_sentences = []
    for sentence in sentences:
        processed = re.sub("[^a-zA-Z]"," ", sentence)
        words = processed.split()
        split_sentences.append(words)

    skip_gram = 1
    workers = multiprocessing.cpu_count()
    bigram_transformer = Phrases(split_sentences)

    model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

    model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
    
    #모델 설계 완료

    #OVERLAP 단어들을 리스트에 넣어서 필터로 만들고
    character_list = [after a while
afterwards
at once
at this moment
at this point
before that
finally
first (second third etc...)
here
in the end
lastly
later on
meanwhile
next
next time
now
on another occasion
previously
since
soon
straightaway
then
until then
when
whenever
while
besides
e.g.
for example
for instance
i.e.
in other words
in that
that is to say
first 
second
firstly 
secondly
first of all
finally
lastly
for one thing
for another
in the first place
to begin with
next
in summation
to conclude
additionally
also
as well
even
furthermore
in addition
indeed
let alone
moreover
not only
accordingly
all the same
an effect of
an outcome of
an upshot of
as a consequence of
as a result of
because
caused by
consequently
despite this
even though
hence
however
in that case
moreover
nevertheless
otherwise
so
so as
stemmed from
still
then
therefore
though
under the circumstances
yet
alternatively
anyway
but
by contrast
differs from
elsewhere
even so
however
in contrast
in fact
in other respects
in spite of this
in that respect
instead
nevertheless
on the contrary
on the other hand
rather
though
whereas
accordingly
as a result
as exemplified by
consequently
for example
for instance
for one thing
including
provided that
since
so
such as
then
therefore
these include
through
unless
without
                     ]
    
    ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
    #우선 토큰화한다.
    retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
    token_input_text = retokenize.tokenize(essay_input_corpus)
    #print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
    #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
    filtered_chr_text = []
    for k in token_input_text:
        for j in character_list:
            if k == j:
                filtered_chr_text.append(j)
    
    #print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
    
    filtered_chr_text_ = set(filtered_chr_text) #중복제거
    filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
    #print (filtered_chr_text__) # 중복값 제거 확인
    
    for i in filtered_chr_text__:
        ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
    
    char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
    char_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수
        
    result_char_ratio = round(char_total_count/total_words * 100, 2)

    #return result_char_ratio, total_sentences, total_words, char_total_count, char_count_, ext_sim_words_key
    return result_char_ratio